### 그레디언트 부스팅 계열의 앙상블 모델

-   대표모델 : XGBoost, LightGBM
-   사용법 숙지 필요


In [84]:
import xgboost, lightgbm

print(f"xgboost : {xgboost.__version__}")
print(f"lightgbm : {lightgbm.__version__}")


xgboost : 1.7.3
lightgbm : 4.1.0


[1] 모듈 로딩 및 데이터 준비 <hr>


In [85]:
## 데이터 파일 변수 선언
feature_name_file = "../data/human_activity/features.txt"
label_file = "../data/human_activity/activity_labels.txt"

x_train_file = "../data/human_activity/train/X_train.txt"
y_train_file = "../data/human_activity/train/y_train.txt"

x_test_file = "../data/human_activity/test/X_test.txt"
y_test_file = "../data/human_activity/test/y_test.txt"


In [86]:
## 데이터 로딩
import pandas as pd
import numpy as np

feature_nameDF = pd.read_csv(
    feature_name_file, sep="\s+", header=None, names=["col_id", "col_name"]
)


In [87]:
feature_nameDF.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 561 entries, 0 to 560
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   col_id    561 non-null    int64 
 1   col_name  561 non-null    object
dtypes: int64(1), object(1)
memory usage: 8.9+ KB


In [88]:
## 중복 체크
feature_nameDF.col_name.duplicated().sum()


84

In [89]:
drop_mask = feature_nameDF[feature_nameDF.col_name.duplicated()].index


In [90]:
feature_nameDF = feature_nameDF.drop(drop_mask, axis=0)


In [91]:
feature_nameDF.reset_index(drop=True, inplace=True)
feature_nameDF.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 477 entries, 0 to 476
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   col_id    477 non-null    int64 
 1   col_name  477 non-null    object
dtypes: int64(1), object(1)
memory usage: 7.6+ KB


In [92]:
x_trainDF = pd.read_csv(x_train_file, sep="\s+", header=None)
y_trainDF = pd.read_csv(y_train_file, sep="\s+", header=None)
x_testDF = pd.read_csv(x_test_file, sep="\s+", header=None)
y_testDF = pd.read_csv(y_test_file, sep="\s+", header=None)


In [93]:
x_trainDF = x_trainDF.drop(drop_mask, axis=1)
x_trainDF = x_trainDF.T.reset_index(drop=True).T
x_trainDF.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7352 entries, 0 to 7351
Columns: 477 entries, 0 to 476
dtypes: float64(477)
memory usage: 26.8 MB


In [94]:
x_testDF = x_testDF.drop(drop_mask, axis=1)
x_testDF = x_testDF.T.reset_index(drop=True).T
x_testDF.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2947 entries, 0 to 2946
Columns: 477 entries, 0 to 476
dtypes: float64(477)
memory usage: 10.7 MB


In [95]:
x_trainDF.columns = feature_nameDF.col_name.tolist()
x_trainDF.head(2)


,tBodyAcc-mean()-X,tBodyAcc-mean()-Y,tBodyAcc-mean()-Z,tBodyAcc-std()-X,tBodyAcc-std()-Y,tBodyAcc-std()-Z,tBodyAcc-mad()-X,tBodyAcc-mad()-Y,tBodyAcc-mad()-Z,tBodyAcc-max()-X,...,fBodyBodyGyroJerkMag-meanFreq(),fBodyBodyGyroJerkMag-skewness(),fBodyBodyGyroJerkMag-kurtosis(),"angle(tBodyAccMean,gravity)","angle(tBodyAccJerkMean),gravityMean)","angle(tBodyGyroMean,gravityMean)","angle(tBodyGyroJerkMean,gravityMean)","angle(X,gravityMean)","angle(Y,gravityMean)","angle(Z,gravityMean)"
0,0.288585,-0.020294,-0.132905,-0.995279,-0.983111,-0.913526,-0.995112,-0.983185,-0.923527,-0.934724,...,-0.074323,-0.298676,-0.710304,-0.112754,0.030400,-0.464761,-0.018446,-0.841247,0.179941,-0.058627
1,0.278419,-0.016411,-0.123520,-0.998245,-0.975300,-0.960322,-0.998807,-0.974914,-0.957686,-0.943068,...,0.158075,-0.595051,-0.861499,0.053477,-0.007435,-0.732626,0.703511,-0.844788,0.180289,-0.054317


In [96]:
x_testDF.columns = feature_nameDF.col_name.tolist()
x_testDF.head(2)


,tBodyAcc-mean()-X,tBodyAcc-mean()-Y,tBodyAcc-mean()-Z,tBodyAcc-std()-X,tBodyAcc-std()-Y,tBodyAcc-std()-Z,tBodyAcc-mad()-X,tBodyAcc-mad()-Y,tBodyAcc-mad()-Z,tBodyAcc-max()-X,...,fBodyBodyGyroJerkMag-meanFreq(),fBodyBodyGyroJerkMag-skewness(),fBodyBodyGyroJerkMag-kurtosis(),"angle(tBodyAccMean,gravity)","angle(tBodyAccJerkMean),gravityMean)","angle(tBodyGyroMean,gravityMean)","angle(tBodyGyroJerkMean,gravityMean)","angle(X,gravityMean)","angle(Y,gravityMean)","angle(Z,gravityMean)"
0,0.257178,-0.023285,-0.014654,-0.938404,-0.920091,-0.667683,-0.952501,-0.925249,-0.674302,-0.894088,...,0.071645,-0.330370,-0.705974,0.006462,0.16292,-0.825886,0.271151,-0.720009,0.276801,-0.057978
1,0.286027,-0.013163,-0.119083,-0.975415,-0.967458,-0.944958,-0.986799,-0.968401,-0.945823,-0.894088,...,-0.401189,-0.121845,-0.594944,-0.083495,0.01750,-0.434375,0.920593,-0.698091,0.281343,-0.083898
